In [1]:
import ast
import boto3
import pandas as pd
import json

In [2]:
label_csv_david = 'D:/canopy_data/csvs/local_multi_label.csv'
label_csv_zhenya = '/Users/purgatorid/Documents/GitHub/canopy-gis/labelling/local_multi_label.csv'

label_df = pd.read_csv(label_csv_zhenya, converters={1:ast.literal_eval})

label_df.head()

,filepaths,labels
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]


# Grab Chip List

In [5]:
bucket = 'canopy-production-ml'

def s3_actions(bucket, in_path=None, out_path=None, copy_list=None, copy_dir=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 
    

    
    if copy_dir or delete or chip_list or chip_count:
        for obj in my_bucket.objects.filter(Prefix=in_path):
            total_files += 1 
            objs.append(obj.key)
            if total_files == stop_at:
                break
        
    tot_objs = len(objs)
        
    if delete:
        for index,obj in enumerate(objs,1):
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            obj.delete()
            
    if chip_list:
        return objs
    
    if chip_count:
        return total_files
    
    if copy_list:
        total_copy_list = len(copy_list)

        for index,obj in enumerate(copy_list,1):
            
            print(f"copying {index} of {total_copy_list}", end='\r', flush=True)
            old_key = in_path + obj
            new_key = out_path + obj
            CopySource = {
                'Bucket': bucket,
                'Key':old_key}
            my_bucket.copy(CopySource, new_key)

In [5]:
path = 'chips/cloudfree-merge-polygons/split/train/'

train_chips = s3_actions(bucket,in_path=path,chip_list=True)

In [7]:
path = 'chips/cloudfree-merge-polygons/split/val/'

val_chips = s3_actions(bucket,in_path=path,chip_list=True)

In [7]:
path = 'chips/cloudfree-merge-polygons/split/test/'

test_chips = s3_actions(bucket,in_path=path,chip_list=True)

In [8]:
len(test_chips)

7707

## make dataframe of s3 filepaths

In [9]:
s3 = boto3.resource('s3')

bucket_name = 'canopy-production-ml'

pc_bucket = s3.Bucket(bucket_name)

print(pc_bucket)

s3.Bucket(name='canopy-production-ml')


In [6]:
train_chips = []

val_chips = []

train_uri = 'chips/cloudfree-merge-polygons/split/train/'

val_uri = 'chips/cloudfree-merge-polygons/split/val/'

for obj in pc_bucket.objects.all():
    #if train_uri in obj.key:
    #    train_chips.append(obj.key)
    if val_uri in obj.key:
        val_chips.append(obj.key)

In [7]:
len(val_chips)

6795

In [8]:
val_chips[0]

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [12]:
df_val_s3 = pd.DataFrame(data={'s3_paths': val_chips})

df_val_s3.shape

(6795, 1)

In [13]:
df_train_s3 = pd.DataFrame(data={'s3_paths': train_chips})

df_train_s3.shape

(27276, 1)

In [10]:
df_test_s3 = pd.DataFrame(data={'s3_paths': test_chips})

df_test_s3.shape

(7707, 1)

## merge dataframes

In [12]:
test_1 = pd.DataFrame(data={'col1': [1,2,3], 'col2': [4,5,6]})

test_2 = pd.DataFrame(data={'col2': [5,6,4], 'col3': [8,9,10]})

test_1.merge(test_2, on='col2')

,col1,col2,col3
0,1,4,10
1,2,5,8
2,3,6,9


In [13]:
label_df.loc[0, 'filepaths']

'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/yes/ISL/100/59/59_800_2800.tif'

In [14]:
df_val_s3.loc[0, 's3_paths']

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [15]:
def get_filename(path):
    return path.split('/')[-1]

label_df['filenames'] = label_df['filepaths'].apply(get_filename)
df_val_s3['filenames'] = df_val_s3['s3_paths'].apply(get_filename)

In [16]:
label_df.head()

,filepaths,labels,lengths,filenames
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_800_2800.tif
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2800.tif
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2900.tif
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_2900.tif
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_3000.tif


In [17]:
df_val_s3.head()

,s3_paths,filenames
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif


In [18]:
merge_df_val = df_val_s3.merge(label_df, on='filenames')

merge_df_val.head()

,s3_paths,filenames,filepaths,labels,lengths
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,"[ISL, Rainforest]",2
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1


In [19]:
assert len(merge_df_val) == len(val_chips)

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

val_ohe = mlb.fit_transform(merge_df_val['labels'])

val_ohe

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [22]:
mlb.classes_

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)

In [21]:
list(val_ohe)[0]

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [23]:
len(val_ohe)

6795

In [26]:
with open('val.lst', 'w') as lst:
    for i in range(len(merge_df_val)):
        lst.write(str(i) + '\t')
        for label in list(val_ohe)[i]:
            lst.write(str(label) + '\t')
        lst.write(merge_df_val.loc[i, 's3_paths'])
        lst.write('\n')

## the function

In [14]:
label_df

,filepaths,labels
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
...,...,...
41766,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41767,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41768,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41769,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]


In [36]:
from sklearn.preprocessing import MultiLabelBinarizer


def make_lst_file(label_df, chips, lst_path):
    s3_df = pd.DataFrame(data={'s3_paths': chips})
    
    def get_filename(path):
        return path.split('/')[-1]

    label_df['filenames'] = label_df['filepaths'].apply(get_filename)
    s3_df['filenames'] = s3_df['s3_paths'].apply(get_filename)
    
    merge_df = s3_df.merge(label_df, on='filenames')
    
    mlb = MultiLabelBinarizer()
    ohe = mlb.fit_transform(merge_df['labels'])

    with open(lst_path, 'w') as lst:
        for i in range(len(merge_df)):
            lst.write(str(i) + '\t')
            for label in list(ohe)[i]:
                lst.write(str(label) + '\t')
            lst.write(merge_df.loc[i, 's3_paths'])
            lst.write('\n')
            
    return mlb.classes_

In [37]:
make_lst_file(label_df, chips, 'train_1.lst')

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)

# Sandbox

In [35]:
def reduce_dominant_class(label_df):
    
    return label_df["labels"]

In [36]:
reduce_dominant_class(label_df)

0                           [ISL]
1                           [ISL]
2                           [ISL]
3                           [ISL]
4                           [ISL]
                   ...           
41766    [Industrial_agriculture]
41767    [Industrial_agriculture]
41768    [Industrial_agriculture]
41769    [Industrial_agriculture]
41770    [Industrial_agriculture]
Name: labels, Length: 41771, dtype: object

In [40]:
assert label_df["labels"][0] == ['ISL']

In [46]:
label_df["labels"].astype('str')

0                           ['ISL']
1                           ['ISL']
2                           ['ISL']
3                           ['ISL']
4                           ['ISL']
                    ...            
41766    ['Industrial_agriculture']
41767    ['Industrial_agriculture']
41768    ['Industrial_agriculture']
41769    ['Industrial_agriculture']
41770    ['Industrial_agriculture']
Name: labels, Length: 41771, dtype: object

In [54]:
# label_df["labels"]

label_df["labels_string"] = label_df["labels"].apply(lambda x : str(x))

# label_df["labels"].applymap(str)

In [13]:
label_df

,filepaths,labels
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
...,...,...
41766,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41767,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41768,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]
41769,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture]


# Create Rainforest Subset DF

In [97]:
label_df_no_rainforest = label_df[~label_df["labels_string"].str.contains('Rainforest')]

In [79]:
label_df_only_multi_rainforest = label_df[(label_df["labels_string"].str.contains("['Rainforest']")) & (label_df["labels_string"] != "['Rainforest']")]

In [90]:
df_only_multi_rainforest = label_df_only_multi_rainforest[label_df_only_multi_rainforest["labels_string"].str.contains('Rainforest')]

In [91]:
df_only_single_rainforest = label_df[label_df["labels_string"] == "['Rainforest']"]

In [102]:
df_only_single_rainforest_subset = df_only_single_rainforest.head(df_only_single_rainforest.shape[0] // 8)

In [105]:
label_df_reduced_rainforest = pd.concat([label_df_no_rainforest, df_only_multi_rainforest,df_only_single_rainforest_subset])

# Export Label File CSV

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer


def export_path_label_csv(label_df, chips, filepath = 'labels_full_train_v1.csv' ):
    s3_df = pd.DataFrame(data={'s3_paths': chips})
    
    def get_filename(path):
        return path.split('/')[-1]

    label_df['filenames'] = label_df['filepaths'].apply(get_filename)
    s3_df['filenames'] = s3_df['s3_paths'].apply(get_filename)
    
    merge_df = s3_df.merge(label_df, on='filenames')
    paths_df = merge_df["s3_paths"] 
    
    mlb = MultiLabelBinarizer()
    ohe = mlb.fit_transform(merge_df['labels'])
    
    ohe_df = pd.DataFrame(ohe)
    
    ohe_df["paths"] = paths_df
    
    ohe_df.to_csv(filepath,index=False)
    
    return ohe_df

In [15]:
ohe_df = export_path_label_csv(label_df,test_chips,"labels_test.csv")

In [16]:
ohe_df.head()

,0,1,2,3,4,5,6,7,8,9,paths
0,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/split/test/100/...
1,0,1,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/split/test/100/...
2,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/split/test/100/...
3,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/split/test/100/...
4,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/split/test/100/...


# Convert to TFRecord File

In [4]:
pd.read_csv("labels_test_v1.csv")

,0,1,2,3,4,5,6,7,8,9,paths
0,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
1,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
2,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
3,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
4,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,0,0,0,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
96,0,1,0,0,0,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
97,0,1,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...
98,0,0,0,0,1,0,0,0,0,0,chips/cloudfree-merge-polygons/sandbox/train/1...


In [74]:
def s3_actions(bucket, in_path=None, out_path=None, copy_list=None, copy_dir=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 
    new_paths = []
    

    
    if copy_dir or delete or chip_list or chip_count:
        for obj in my_bucket.objects.filter(Prefix=in_path):
            total_files += 1 
            objs.append(obj.key)
            if total_files == stop_at:
                break
        
    tot_objs = len(objs)
        
    if delete:
        for index,obj in enumerate(objs,1):
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            obj.delete()
            
    if chip_list:
        return objs
    
    if chip_count:
        return total_files
    
    if copy_list:
        total_copy_list = len(copy_list)

        for index,obj in enumerate(copy_list,1):
            
            print(f"copying {index} of {total_copy_list}", end='\r', flush=True)
            old_key = obj
            polygon_id = obj.split("/")[-1].split("_")[0]
            filename = obj.split('/')[-1]
            new_key = f'{out_path}{polygon_id}/{filename}'
            new_paths.append(new_key)
    return new_paths
            
#             CopySource = {
#                 'Bucket': bucket,
#                 'Key':old_key}
#             my_bucket.copy(CopySource, new_key)
            
            
            
                        
            
            

In [75]:
bucket = 'canopy-production-ml'
in_path = 'chips/cloudfree-merge-polygons/split/train/'
out_path = 'chips/cloudfree-merge-polygons/sandbox/train/'

new_paths = s3_actions(bucket,in_path=in_path,out_path=out_path,copy_list=train_copy)

In [77]:
len(new_paths)

100

In [78]:
df_labels_paths["paths"] = new_paths

In [81]:
df_labels_paths.to_csv("labels_test_v1.csv",index=False)

# Generate Class Weights

In [122]:
labels = {}

for l_l in label_df["labels"]:
    
    for l in l_l:
        
        if l in labels.keys():
            
            labels[l] += 1
            
        else:
            
            labels[l] = 1
    

In [129]:
labels

{'ISL': 6963,
 'Rainforest': 25687,
 'Roads': 1476,
 'Shifting_cultivation': 3900,
 'Savannah': 4136,
 'River': 2626,
 'Habitation': 808,
 'Mining': 219,
 'Industrial_agriculture': 1462,
 'Water': 1851}

In [132]:
labels_sum = sum(labels.values())
keys_len = len(labels.keys())

In [137]:
# (1 / labels["ISL"]) * (labels_sum)/keys_len

class_weight = {}

for label_name in labels.keys():
    class_weight[label_name] = (1 / labels[label_name]) * (labels_sum)/keys_len
    

In [138]:
class_weight

{'ISL': 0.7055579491598449,
 'Rainforest': 0.19125627749445245,
 'Roads': 3.3284552845528452,
 'Shifting_cultivation': 1.2596923076923077,
 'Savannah': 1.1878143133462282,
 'River': 1.8708301599390709,
 'Habitation': 6.080198019801981,
 'Mining': 22.432876712328767,
 'Industrial_agriculture': 3.360328317373461,
 'Water': 2.654132901134522}

In [4]:
def generate_class_weight(csv_loc,labels_loc):
    
    df = pd.read_csv(csv_loc)
    
    with open(labels_loc) as jsonfile:
        data = json.load(jsonfile)
    
    num_labels = len(data['label_names'].keys())
    
    labels = {}
    
    for column in df.columns[0:num_labels]:
        col_count = df[column].value_counts()[1]
        labels[column] = col_count

    labels_sum = sum(labels.values())
    keys_len = len(labels.keys())

    class_weight = {}

    for label_name in labels.keys():
        class_weight[label_name] = (1 / labels[label_name]) * (labels_sum)/keys_len

    return class_weight
    
    

In [5]:
generate_class_weight('labels_full_train_v1.csv','labels.json')

{'0': 4.8802488335925345,
 '1': 0.8312582781456953,
 '2': 2.7406113537117904,
 '3': 21.944055944055943,
 '4': 0.19916222391469918,
 '5': 1.8869512928442576,
 '6': 3.5821917808219177,
 '7': 1.0633683497119621,
 '8': 1.0302035456336178,
 '9': 2.2706222865412444}

In [275]:
data

{'label_names': {'1': 'Habitation',
  '2': 'ISL',
  '3': 'Industrial_agriculture',
  '4': 'Mining',
  '5': 'Rainforest',
  '6': 'River',
  '7': 'Roads',
  '8': 'Savannah',
  '9': 'Shifting_cultivation',
  '10': 'Water'}}

In [264]:
# column_rename

for column in df.columns[0:len(column_rename.keys())]:
    print(df[column].value_counts()[1])

643
3775
1145
143
15756
1663
876
2951
3046
1382
